In [22]:
import numpy as np
from keras import backend as K
from keras.models import load_model

In [10]:
def get_layer_i_output(model, i, data):
    layer_model = K.function([model.layers[0].input], [model.layers[i].output])
    ret = layer_model([data])[0]
    num = data.shape[0]
    ret = np.reshape(ret, (num, -1))
    return ret

In [19]:
def load_data(pathToData):
    x_train = np.load(pathToData + 'x_train.npy')
    y_train = np.load(pathToData + 'y_train.npy')
    x_test = np.load(pathToData  + 'x_test.npy')
    y_test = np.load(pathToData  + 'y_test.npy')
    return x_train, y_train, x_test, y_test

# NC: Neuron Coverage

In [41]:
class Coverage:
    def __init__(self, model, x_train, y_train, x_test, y_test, x_adv):
        self.model = model
        self.x_train = x_train
        self.y_train = y_train
        self.x_test = x_test
        self.y_test = y_test
        self.x_adv = x_adv

    # find scale factors and min num
    def scale(self, layers, batch=1024):
        data_num = self.x_adv.shape[0]
        factors = dict()
        for i in layers:
            begin, end = 0, batch
            max_num, min_num = np.NINF, np.inf
            while begin < data_num:
                layer_output = get_layer_i_output(self.model, i, self.x_adv[begin:end])
                tmp = layer_output.max()
                max_num = tmp if tmp > max_num else max_num
                tmp = layer_output.min()
                min_num = tmp if tmp < min_num else min_num
                begin += batch
                end += batch
            factors[i] = (max_num - min_num, min_num)
        return factors

    # 1 Neuron Coverage
    def NC(self, layers, threshold=0., batch=1024):
        factors = self.scale(layers, batch=batch)
        neuron_num = 0
        for i in layers:
            out_shape = self.model.layers[i].output.shape
            neuron_num += np.prod(out_shape[1:])
        neuron_num = int(neuron_num)

        activate_num = 0
        data_num = self.x_adv.shape[0]
        for i in layers:
            neurons = np.prod(self.model.layers[i].output.shape[1:])
            buckets = np.zeros(neurons).astype('bool')
            begin, end = 0, batch
            while begin < data_num:
                layer_output = get_layer_i_output(self.model, i, self.x_adv[begin:end])
                # scale the layer output to (0, 1)
                layer_output -= factors[i][1]
                layer_output /= factors[i][0]
                col_max = np.max(layer_output, axis=0)
                begin += batch
                end += batch
                buckets[col_max > threshold] = True
            activate_num += np.sum(buckets)
        #print('NC:\t{:.3f} activate_num:\t{} neuron_num:\t{}'.format(activate_num / neuron_num, activate_num, neuron_num))
        return activate_num / neuron_num, activate_num, neuron_num
    
    # 2 k-multisection neuron coverage, neuron boundary coverage and strong activation neuron coverage
    def KMNC(self, layers, k=10, batch=1024):
        neuron_num = 0
        for i in layers:
            out_shape = self.model.layers[i].output.shape
            neuron_num += np.prod(out_shape[1:])
        neuron_num = int(neuron_num)

        covered_num = 0
        l_covered_num = 0
        u_covered_num = 0
        for i in layers:
            neurons = np.prod(self.model.layers[i].output.shape[1:])
            print(neurons)
            begin, end = 0, batch
            data_num = self.x_train.shape[0]

            neuron_max = np.full(neurons, np.NINF).astype('float')
            neuron_min = np.full(neurons, np.inf).astype('float')
            while begin < data_num:
                layer_output_train = get_layer_i_output(self.model, i, self.x_train[begin:end])
                batch_neuron_max = np.max(layer_output_train, axis=0)
                batch_neuron_min = np.min(layer_output_train, axis=0)
                neuron_max = np.maximum(batch_neuron_max, neuron_max)
                neuron_min = np.minimum(batch_neuron_min, neuron_min)
                begin += batch
                end += batch
            buckets = np.zeros((neurons, k + 2)).astype('bool')
            interval = (neuron_max - neuron_min) / k
            # print(interval[8], neuron_max[8], neuron_min[8])
            begin, end = 0, batch
            data_num = self.x_adv.shape[0]
            while begin < data_num:
                layer_output_adv = get_layer_i_output(self.model, i, self.x_adv[begin: end])
                layer_output_adv -= neuron_min
                layer_output_adv /= (interval + 10 ** (-100))
                layer_output_adv[layer_output_adv < 0.] = -1
                layer_output_adv[layer_output_adv >= k / 1.0] = k
                layer_output_adv = layer_output_adv.astype('int')
                # index 0 for lower, 1 to k for between, k + 1 for upper
                layer_output_adv = layer_output_adv + 1
                for j in range(neurons):
                    uniq = np.unique(layer_output_adv[:, j])
                    # print(layer_output_adv[:, j])
                    buckets[j, uniq] = True
                begin += batch
                end += batch
            covered_num += np.sum(buckets[:, 1:-1])
            u_covered_num += np.sum(buckets[:, -1])
            l_covered_num += np.sum(buckets[:, 0])
        print('KMNC:\t{:.3f} covered_num:\t{}'.format(covered_num / (neuron_num * k), covered_num))
        print(
            'NBC:\t{:.3f} l_covered_num:\t{}'.format((l_covered_num + u_covered_num) / (neuron_num * 2), l_covered_num))
        print('SNAC:\t{:.3f} u_covered_num:\t{}'.format(u_covered_num / neuron_num, u_covered_num))
        return covered_num / (neuron_num * k), (l_covered_num + u_covered_num) / (
                    neuron_num * 2), u_covered_num / neuron_num, covered_num, l_covered_num, u_covered_num, neuron_num * k

    # 3 top-k neuron coverage
    def TKNC(self, layers, k=2, batch=1024):
        def top_k(x, k):
            ind = np.argpartition(x, -k)[-k:]
            return ind[np.argsort((-x)[ind])]

        neuron_num = 0
        for i in layers:
            out_shape = self.model.layers[i].output.shape
            neuron_num += np.prod(out_shape[1:])
        neuron_num = int(neuron_num)

        pattern_num = 0
        data_num = self.x_adv.shape[0]
        for i in layers:
            pattern_set = set()
            begin, end = 0, batch
            while begin < data_num:
                layer_output = get_layer_i_output(self.model, i, self.x_adv[begin:end])
                topk = np.argpartition(layer_output, -k, axis=1)[:, -k:]
                topk = np.sort(topk, axis=1)
                # or in order
                # topk = np.apply_along_axis[lambda x: top_k(layer_output, k), 1, layer_output]
                for j in range(topk.shape[0]):
                    pattern_set.add(tuple(topk[j]))
                begin += batch
                end += batch
            pattern_num += len(pattern_set)
        print(
            'TKNC:\t{:.3f} pattern_num:\t{} neuron_num:\t{}'.format(pattern_num / neuron_num, pattern_num, neuron_num))
        return pattern_num / neuron_num, pattern_num, neuron_num

    # 4 top-k neuron patterns
    def TKNP(self, layers, k=2, batch=1024):
        def top_k(x, k):
            ind = np.argpartition(x, -k)[-k:]
            return ind[np.argsort((-x)[ind])]

        def to_tuple(x):
            l = list()
            for row in x:
                l.append(tuple(row))
            return tuple(l)

        pattern_set = set()
        layer_num = len(layers)
        data_num = self.x_adv.shape[0]
        patterns = np.zeros((data_num, layer_num, k))
        layer_cnt = 0
        for i in layers:
            neurons = np.prod(self.model.layers[i].output.shape[1:])
            begin, end = 0, batch
            while begin < data_num:
                layer_output = get_layer_i_output(self.model, i, self.x_adv[begin:end])
                topk = np.argpartition(layer_output, -k, axis=1)[:, -k:]
                topk = np.sort(topk, axis=1)
                # or in order
                # topk = np.apply_along_axis[lambda x: top_k(layer_output, k), 1, layer_output]
                patterns[begin:end, layer_cnt, :] = topk
                begin += batch
                end += batch
            layer_cnt += 1

        for i in range(patterns.shape[0]):
            pattern_set.add(to_tuple(patterns[i]))
        pattern_num = len(pattern_set)
        print('TKNP:\t{:.3f}'.format(pattern_num))
        return pattern_num

In [42]:
## Load benign images from mnist, cifar, or svhn
pathToData = "/home/bouchekir/Desktop/NN_Tesing/data/mnist_data/mnist_"
x_train, y_train, x_test, y_test = load_data(pathToData)

In [43]:
# Load the model 
modelPath = '/home/bouchekir/Desktop/NN_Tesing/data/mnist_data/model/lenet1.h5'
model = load_model(modelPath)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 24, 24, 4)         104       
                                                                 
 activation_1 (Activation)   (None, 24, 24, 4)         0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 12, 12, 4)        0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 8, 8, 12)          1212      
                                                                 
 activation_2 (Activation)   (None, 8, 8, 12)          0         
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 4, 4, 12)         0         
 2D)                                                  

In [44]:
adv_dir = '/home/bouchekir/Desktop/NN_Tesing/data/mnist_data/model/adv_lenet1_PGD.npy'
x_adv = np.load(adv_dir)
print(len(x_adv))

10000


In [46]:
l = [0, 8]
for i in range(1, len(x_adv), 200):
    if i == 1000 or i == 3000 or i == 5000 or i == 7000 or i == 9000:
         print("i:", i)
    coverage = Coverage(model, x_train, y_train, x_test, y_test, x_adv[:i])
    nc1, _, _ = coverage.NC(l, threshold=0.3)
    nc2, _, _ = coverage.NC(l, threshold=0.5)
    kmnc, nbc, snac, _, _, _, _ = coverage.KMNC(l)
    tknc, _, _ = coverage.TKNC(l)
    tknp = coverage.TKNP(l)
    
    
    print("\n------------------------------------------------------------------------------\n")
    print('x: {}   \n'.format(i))
    print('NC(0.1): {}   \n'.format(nc1))
    print('NC(0.3): {}   \n'.format(nc2))
    print('TKNC: {}   \n'.format(tknc))
    print('TKNP: {} \n'.format(tknp))
    print('KMNC: {} \n'.format(kmnc))
    print('NBC: {}  \n'.format(nbc))
    print('SNAC: {} \n'.format(snac))

2304
10
KMNC:	0.099 covered_num:	2301
NBC:	0.003 l_covered_num:	5
SNAC:	0.003 u_covered_num:	8
TKNC:	0.001 pattern_num:	2 neuron_num:	2314
TKNP:	1.000

------------------------------------------------------------------------------

x: 1   

NC(0.1): 0.9382022471910112   

NC(0.3): 0.3630077787381158   

TKNC: 0.000864304235090752   

TKNP: 1 

KMNC: 0.09943820224719101 

NBC: 0.0028089887640449437  

SNAC: 0.003457216940363008 

2304
10
KMNC:	0.683 covered_num:	15811
NBC:	0.020 l_covered_num:	39
SNAC:	0.023 u_covered_num:	54
TKNC:	0.100 pattern_num:	231 neuron_num:	2314
TKNP:	201.000

------------------------------------------------------------------------------

x: 201   

NC(0.1): 1.0   

NC(0.3): 1.0   

TKNC: 0.09982713915298184   

TKNP: 201 

KMNC: 0.683275713050994 

NBC: 0.020095073465859984  

SNAC: 0.023336214347450302 

2304
10
KMNC:	0.737 covered_num:	17053
NBC:	0.022 l_covered_num:	47
SNAC:	0.025 u_covered_num:	57
TKNC:	0.182 pattern_num:	420 neuron_num:	2314
TKNP:	399.000

2304
10
KMNC:	0.874 covered_num:	20234
NBC:	0.043 l_covered_num:	102
SNAC:	0.042 u_covered_num:	97
TKNC:	1.249 pattern_num:	2890 neuron_num:	2314
TKNP:	3673.000

------------------------------------------------------------------------------

x: 4001   

NC(0.1): 1.0   

NC(0.3): 1.0   

TKNC: 1.2489196197061365   

TKNP: 3673 

KMNC: 0.8744165946413137 

NBC: 0.04299913569576491  

SNAC: 0.04191875540190147 

2304
10
KMNC:	0.877 covered_num:	20296
NBC:	0.043 l_covered_num:	104
SNAC:	0.042 u_covered_num:	97
TKNC:	1.303 pattern_num:	3014 neuron_num:	2314
TKNP:	3844.000

------------------------------------------------------------------------------

x: 4201   

NC(0.1): 1.0   

NC(0.3): 1.0   

TKNC: 1.3025064822817631   

TKNP: 3844 

KMNC: 0.8770959377700951 

NBC: 0.043431287813310286  

SNAC: 0.04191875540190147 

2304
10
KMNC:	0.880 covered_num:	20369
NBC:	0.044 l_covered_num:	105
SNAC:	0.042 u_covered_num:	97
TKNC:	1.349 pattern_num:	3121 neuron_num:	2314
TKNP:	4013.000

-----------

2304
10
KMNC:	0.911 covered_num:	21070
NBC:	0.055 l_covered_num:	141
SNAC:	0.048 u_covered_num:	112
TKNC:	2.185 pattern_num:	5057 neuron_num:	2314
TKNP:	6944.000

------------------------------------------------------------------------------

x: 8001   

NC(0.1): 1.0   

NC(0.3): 1.0   

TKNC: 2.1853932584269664   

TKNP: 6944 

KMNC: 0.9105445116681071 

NBC: 0.05466724286949006  

SNAC: 0.04840103716508211 

2304
10
KMNC:	0.911 covered_num:	21088
NBC:	0.056 l_covered_num:	144
SNAC:	0.049 u_covered_num:	113
TKNC:	2.228 pattern_num:	5156 neuron_num:	2314
TKNP:	7094.000

------------------------------------------------------------------------------

x: 8201   

NC(0.1): 1.0   

NC(0.3): 1.0   

TKNC: 2.2281763180639587   

TKNP: 7094 

KMNC: 0.9113223854796888 

NBC: 0.05553154710458081  

SNAC: 0.04883318928262748 

2304
10
KMNC:	0.913 covered_num:	21117
NBC:	0.056 l_covered_num:	146
SNAC:	0.049 u_covered_num:	113
TKNC:	2.271 pattern_num:	5255 neuron_num:	2314
TKNP:	7249.000

---------